In [1]:
import pandas as pd
import numpy as np

In [15]:
df1 = pd.read_csv('2024_feb_unduplicated_orders.csv')
len(df1['title'].value_counts())

3926

In [5]:
df1.shape

(58958, 22)

In [11]:
df = pd.read_csv('product_data_status.csv')
df.head()

,Main Reference,Product Name,Brand,Attributes,Product Reference,Product Model,Sale Price,Regular Price,RRP Price,Categories,...,Product Specific Images,On Sale,Active,Discontinued,Meta Title,Meta Keywords,Meta Description,Weight,Friendly URL,Image Location
0,1405,Leather Shirt Jacket Birmingham Black/ant,Ashwood,Size: XS,140502,DM-SHIRT,99.0,189.0,0.0,"&middot;&nbsp;Mens,Leather Jackets,Offers,Sale...",...,5,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/mens-c4/leath...,https://www.leathercompany.co.uk/images/produc...
1,1405,Leather Shirt Jacket Birmingham Black/ant,Ashwood,Size: S,140503,DM-SHIRT,99.0,189.0,0.0,"&middot;&nbsp;Mens,Leather Jackets,Offers,Sale...",...,5,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/mens-c4/leath...,https://www.leathercompany.co.uk/images/produc...
2,1405,Leather Shirt Jacket Birmingham Black/ant,Ashwood,Size: M,140504,DM-SHIRT,99.0,189.0,0.0,"&middot;&nbsp;Mens,Leather Jackets,Offers,Sale...",...,5,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/mens-c4/leath...,https://www.leathercompany.co.uk/images/produc...
3,1405,Leather Shirt Jacket Birmingham Black/ant,Ashwood,Size: L,140505,DM-SHIRT,99.0,189.0,0.0,"&middot;&nbsp;Mens,Leather Jackets,Offers,Sale...",...,5,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/mens-c4/leath...,https://www.leathercompany.co.uk/images/produc...
4,1405,Leather Shirt Jacket Birmingham Black/ant,Ashwood,Size: XL,140506,DM-SHIRT,99.0,189.0,0.0,"&middot;&nbsp;Mens,Leather Jackets,Offers,Sale...",...,5,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/mens-c4/leath...,https://www.leathercompany.co.uk/images/produc...


In [12]:
df.shape

(6900, 24)

In [13]:
df['FinalCategory'] = df['Categories'].str.split(',').str[-1].str.replace('<br />', '').str.replace('<b', '')
len(df['FinalCategory'].value_counts())

59

In [44]:
df[df['Product Name'].str.contains("Super Bomber")]

,Main Reference,Product Name,Brand,Attributes,Product Reference,Product Model,Sale Price,Regular Price,RRP Price,Categories,...,On Sale,Active,Discontinued,Meta Title,Meta Keywords,Meta Description,Weight,Friendly URL,Image Location,FinalCategory


In [45]:
df['Product Name'] = df['Product Name'].str.lstrip()
df['Product Name'] = df['Product Name'].str.rstrip()
len(df['Product Name'].value_counts())

1500

In [46]:
df['Product Name'].value_counts()

Product Name
Toscana 3/4 Shearling Coat Tan : Octavia                                            36
Long Length Toscana Suede Leather Coat Black : Alaska                               36
Leather Jacket Timber : Bacchus                                                     28
Sheepskin Coat Brown : Hektor                                                       14
Leather Flying Jacket Black : Welkin                                                14
                                                                                    ..
Unisex Travel And Shopper Bag Blue : 00650641P 2K                                    1
Unisex Shopper Work Bag Brown : 0617251P                                             1
Unisex Shopper Work Bag Brown : 0650041P                                             1
Unisex Travel and Shopper Bag Brown : 0650641P                                       1
The Bridge B Go Italian 4 Wheeled Trolley Case Black/Gunmetal  - 071346 3L 5P NH     1
Name: count, Length: 1500, dty

In [37]:
df[df['Product Name'].str.contains("Toscana 3/4 Shearling Coat Tan : Octavia")].drop_duplicates()

,Main Reference,Product Name,Brand,Attributes,Product Reference,Product Model,Sale Price,Regular Price,RRP Price,Categories,...,On Sale,Active,Discontinued,Meta Title,Meta Keywords,Meta Description,Weight,Friendly URL,Image Location,FinalCategory
3360,7053,Toscana 3/4 Shearling Coat Tan : Octavia,Ashwood,Size: 6,705300,B-5661,499.0,795.0,0.0,"&middot;&nbsp;Ladies,Sheepskin Jackets,Ladies ...",...,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/ladies-c3/she...,https://www.leathercompany.co.uk/images/produc...,Sale For Her
3361,7053,Toscana 3/4 Shearling Coat Tan : Octavia,Ashwood,Size: 8,705301,B-5661,499.0,795.0,0.0,"&middot;&nbsp;Ladies,Sheepskin Jackets,Ladies ...",...,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/ladies-c3/she...,https://www.leathercompany.co.uk/images/produc...,Sale For Her
3362,7053,Toscana 3/4 Shearling Coat Tan : Octavia,Ashwood,Size: 10,705302,B-5661,499.0,795.0,0.0,"&middot;&nbsp;Ladies,Sheepskin Jackets,Ladies ...",...,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/ladies-c3/she...,https://www.leathercompany.co.uk/images/produc...,Sale For Her
3363,7053,Toscana 3/4 Shearling Coat Tan : Octavia,Ashwood,Size: 12,705303,B-5661,499.0,795.0,0.0,"&middot;&nbsp;Ladies,Sheepskin Jackets,Ladies ...",...,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/ladies-c3/she...,https://www.leathercompany.co.uk/images/produc...,Sale For Her
3364,7053,Toscana 3/4 Shearling Coat Tan : Octavia,Ashwood,Size: 14,705304,B-5661,499.0,795.0,0.0,"&middot;&nbsp;Ladies,Sheepskin Jackets,Ladies ...",...,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/ladies-c3/she...,https://www.leathercompany.co.uk/images/produc...,Sale For Her
3365,7053,Toscana 3/4 Shearling Coat Tan : Octavia,Ashwood,Size: 16,705305,B-5661,499.0,795.0,0.0,"&middot;&nbsp;Ladies,Sheepskin Jackets,Ladies ...",...,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/ladies-c3/she...,https://www.leathercompany.co.uk/images/produc...,Sale For Her
3366,7053,Toscana 3/4 Shearling Coat Tan : Octavia,Ashwood,Size: 18,705306,B-5661,499.0,795.0,0.0,"&middot;&nbsp;Ladies,Sheepskin Jackets,Ladies ...",...,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/ladies-c3/she...,https://www.leathercompany.co.uk/images/produc...,Sale For Her
3367,7053,Toscana 3/4 Shearling Coat Tan : Octavia,Ashwood,Size: 20,705307,B-5661,499.0,795.0,0.0,"&middot;&nbsp;Ladies,Sheepskin Jackets,Ladies ...",...,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/ladies-c3/she...,https://www.leathercompany.co.uk/images/produc...,Sale For Her
3368,7053,Toscana 3/4 Shearling Coat Tan : Octavia,Ashwood,Size: 22,705308,B-5661,499.0,795.0,0.0,"&middot;&nbsp;Ladies,Sheepskin Jackets,Ladies ...",...,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/ladies-c3/she...,https://www.leathercompany.co.uk/images/produc...,Sale For Her
3369,7053,Toscana 3/4 Shearling Coat Tan : Octavia,Ashwood,Size: 24,705309,B-5661,499.0,795.0,0.0,"&middot;&nbsp;Ladies,Sheepskin Jackets,Ladies ...",...,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/ladies-c3/she...,https://www.leathercompany.co.uk/images/produc...,Sale For Her
